# Celebrity Face Generation

### This kernel uses Generative Adversarial Networks (GAN) i.e. DCGAN here to generate faces. The faces are generated using a generator and a discriminator. Both these models are written using Keras model inheritence which helps to separate out blocks and reuse the blocks in multiple places

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q imageio
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import imageio
import pathlib
import time
import PIL
import glob
from IPython import display

import tensorflow as tf
from tensorflow.keras import layers
tf.__version__

## Data Preprocessing

Here we can a sample image to understand the images in the dataset

In [ ]:
test_image = np.array(Image.open('../input/celeba-dataset/img_align_celeba/img_align_celeba/000023.jpg'))
print(test_image.shape)
plt.imshow(test_image)

Loading the directory of the image

In [ ]:
data_dir = pathlib.Path('../input/celeba-dataset/img_align_celeba/img_align_celeba/')
image_count = len(list(data_dir.glob('*.jpg')))
print("Total images : ",image_count)

Some of the variable are used so that the code can be easily tuned if we need to conduct multiple experiments.

### How did I choose the height of width of image and also the generator ?
- Having to train on full image has better advantages in generating better faces. I did try to train on full images. But due to the memory constraints and also the limited training time, the image width and height was taken. The idea was to approximately reduce image size by half.
- Reducing image height and width by half would give the new image shape to be (109, 89 , 3). This should have been the size of the image which it should have been resized to.
- But I had an approximate idea that, in my generators I would upsample the initial random input 3 times. So since the height and width of the inputs double on each upsample (strides=2), 3 upsample means the image increases its height and width 8 times. So the final output of the generator should have height and width divisible by 8. Hence I chose (104,88,3) as the input shape. The height and width are the nearest multiples of 8 which are lesser than my target i.e. 109 and 89.
- So now the generator's initial height and width should be 8 times lesser than final output i.e
 - generator_height = final_image_height/8 = 104/8 = 13
 - generator_width = final_image_width/8 = 88/8 = 11

In [ ]:
IMAGE_HEIGHT = 104
IMAGE_WIDTH = 88
GENERATOR_IMAGE_HEIGHT = 13
GENERATOR_IMAGE_WIDTH = 11
NOISE_DIM = 100
VAL_SPLIT = 0.2

Tensorflow's Dataset module has been used because the images are fetched on the go and the module takes care of the corner cases. The noise dimension is basically the input noise length that is fed to the generator

Even though we do not need validation set actually for generation, We choose validation dataset to be 20% of the total just to keep an open option on a scenario where we might need to use the unseen dataset. Also, reducing training size also decreased training time :) 

In [ ]:
list_data = tf.data.Dataset.list_files(str(data_dir/'*.jpg'), shuffle=False)
list_data = list_data.shuffle(image_count,reshuffle_each_iteration=False)

In [ ]:
for f in list_data.take(5):
    print(f.numpy())

In [ ]:
val_size = int(image_count*VAL_SPLIT)
train_ds_list = list_data.skip(val_size)
val_ds_list = list_data.take(val_size)

In [ ]:
# A funtion to plot the preprocessed image
def plot_image(processed_image):
    processed_image = processed_image * 255
    img = tf.cast(processed_image, dtype=tf.uint8)
    plt.imshow(img)

**The below function is used to fetch images while we are dynamically fetching images during training**

In [ ]:
def get_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, [IMAGE_HEIGHT, IMAGE_WIDTH])
    img = img / 255
    return img

plot_image(get_image('../input/celeba-dataset/img_align_celeba/img_align_celeba/000026.jpg'))

In [ ]:
train_ds = train_ds_list.map(get_image)
val_ds = val_ds_list.map(get_image)

**The batch size is chosen considering memory limitation. Batch size of more than 32 would fail to allocate necessary memory**

In [ ]:
BATCH_SIZE = 32

#### Tensorflow offers configuration to prefetch images  to save image loading time. The below function optimizes the overall training performance

In [ ]:
def configure_for_performance(ds):
#     ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

## Generator and Discriminator models

#### This is one block of Upsampling done in the generator model. Each Conv2DTranspose layer here is upsampling by a factor of 2

In [ ]:
class Upsample(tf.keras.Model):
    def __init__(self, filters, kernel, stride ):
        super(Upsample,self).__init__()
        self.filters = filters
        self.conv2DT = layers.Conv2DTranspose(self.filters, kernel_size = (kernel,kernel), strides=(stride,stride), padding='same', use_bias = 'false')
        self.batchnorm = layers.BatchNormalization()
        self.lrelu = layers.LeakyReLU()
    
    def call(self, inputs):
        # input shape - (batch_size, x , y, z)
        x = self.conv2DT(inputs)
        x = self.batchnorm(x)
        out = self.lrelu(x)
        return out
        # output shape - (batch_size, 2x , 2y, self.filters)

In [ ]:
# For the sake of understanding consider initial filters to be 256 and generator image height and generator image width to be 13 and 11 respectively 
# and follow the comments
class Generator(tf.keras.Model):
    #                   13      11      256          5
    def __init__(self,height, width, init_filters, kernel):
        super(Generator,self).__init__()
        self.dense = layers.Dense(height*width*init_filters, use_bias=False, input_shape=(NOISE_DIM,))
        self.batchnorm = layers.BatchNormalization()
        self.lrelu = layers.LeakyReLU()
        self.reshape = layers.Reshape((height, width, init_filters))
        self.upsample1 = Upsample(init_filters,kernel,1)
        self.upsample2 = Upsample(init_filters/2,kernel,2)
        self.upsample3 = Upsample(init_filters/4,kernel,2)
        self.convtranspose = layers.Conv2DTranspose(3,(kernel,kernel), strides=(2,2), use_bias='false',padding='same', activation='sigmoid')
        
    def call(self, inputs):
        # input shape - (batch_size, 100)
        x = self.dense(inputs)
        # shape - (batch_size, 13*11*256)
        x = self.batchnorm(x)
        x = self.lrelu(x)
        x = self.reshape(x)
        # shape - (batch_size, 13, 11, 256)
        x = self.upsample1(x)
        # shape - (batch_size, 13, 11, 256)
        x = self.upsample2(x)
        # shape - (batch_size, 26, 22, 128)
        x = self.upsample3(x)
        # shape - (batch_size, 52, 44, 64)
        out = self.convtranspose(x)
        # shape - (batch_size, 104, 88, 3)
        return out

In [ ]:
generator = Generator(GENERATOR_IMAGE_HEIGHT,GENERATOR_IMAGE_WIDTH,256,5)

In [ ]:
# Testing how generator generates now
noise = tf.random.normal([1,NOISE_DIM])
generated_image = generator(noise,training=False)
print("Image Shape: ",generated_image.shape)
plt.imshow(generated_image[0])

In [ ]:
generator.build((1,100))
generator.summary()

In [ ]:
# This is one CNN block during downnsampling
class CNNBlock(tf.keras.Model):
    def __init__(self, filters, kernel, stride):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(filters, kernel_size = (kernel,kernel), strides = (stride, stride), padding='same')
        self.lrelu = layers.LeakyReLU()
        self.dropout = layers.Dropout(0.3)
        
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.lrelu(x)
        out = self.dropout(x)
        return out

In [ ]:
# For the sake of understanding consider initial filters to be 32 and image height and image width to be 104 and 88 respectively 
# and follow the comments
class Discriminator(tf.keras.Model):
    #                      32          5
    def __init__(self, init_filters, kernel):
        super(Discriminator,self).__init__()
        self.input_layer = layers.InputLayer(input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH,3))
        self.cnn1 = CNNBlock(init_filters, kernel, 1)
        self.cnn2 = CNNBlock(init_filters*2, kernel, 2)
        self.cnn3 = CNNBlock(init_filters*4, kernel, 1)
        self.cnn4 = CNNBlock(init_filters*8, kernel, 1)
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(1)
        
    def call(self, inputs):
        # shape - (batch_size, 104, 88,3)
        x = self.input_layer(inputs)
        # shape - (batch_size, 104, 88,3)
        x = self.cnn1(x)
        # shape - (batch_size, 104, 88, 32)
        x = self.cnn2(x)
        # shape - (batch_size, 52, 44, 64)
        x = self.cnn3(x)
        # shape - (batch_size, 52, 44, 128)
        x = self.cnn4(x)
        # shape - (batch_size, 52, 44, 256)
        x = self.flatten(x)
        # shape - (batch_size, 52 * 44 * 256)
        out = self.dense(x)
        # shape - (batch_size, 1)
        return out

In [ ]:
discriminator = Discriminator(32, 5)

In [ ]:
discriminator.build((BATCH_SIZE,IMAGE_HEIGHT, IMAGE_WIDTH,3))
discriminator.summary()

#### The discriminator is punished if the prediction is not near to 1 for real images and if the prediction is not 0 for fake/generator images. This is basically to improve discriminator to learn generated images
#### But for the generator the idea is to trick the discriminator. So for a generated image, if the descriminator predicts it to be 1 that means generator has won. So we pass the generated output through discriminator and punish the generator if the output from discriminator for the generated image is not near to 1

In [ ]:
#Loss functions for generator and discriminator
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    return real_loss + fake_loss

In [ ]:
#Optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

### Epochs and function for image generation

In [ ]:
EPOCHS = 40
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, NOISE_DIM])

In [ ]:
seed = tf.random.normal([num_examples_to_generate, NOISE_DIM])

In [ ]:
# This is used to generate images during training and saving them for final GIF
def generate_and_save_images(model, epoch,step, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        image = tf.cast(predictions[i, :, :]*255.0,dtype=tf.uint8)
        plt.imshow(image)
        plt.axis('off')
    
    plt.savefig('image_at_epoch_{:04d}_step_{:06d}.png'.format(epoch, step))
    plt.show()

## Training

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE,NOISE_DIM])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        fake_output = discriminator(generated_images, training=True)
        real_output = discriminator(images, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        
    gradients_of_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_disc= disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_gen, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_disc, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        
        for i,image_batch in enumerate(dataset):
            train_step(image_batch)
#             if (i%100==0):
#                 display.clear_output(wait=True)
#                 print("{}th batch done".format(i))
#             if (i%500==0):
#                 display.clear_output(wait=True)
#                 generate_and_save_images(generator, epoch, i, seed)
        
        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch, 9999, seed)
        
#         checkpoint.save(file_prefix = checkpoint_prefix)
        generator.save_weights('generator_weights_epoch_{}'.format(epoch))
            
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    display.clear_output(wait=True)
    generate_and_save_images(generator, epoch, 55000, seed)

In [ ]:
# I use this section to clear the GPU memory so that I can reuse GPU again
# !pip install GPUtil

# from GPUtil import showUtilization as gpu_usage
# gpu_usage()

# import torch
# torch.cuda.empty_cache()

# gpu_usage()
# from numba import cuda
# cuda.select_device(0)
# cuda.close()
# gpu_usage()

In [ ]:
start = time.time()
train(train_ds, EPOCHS)
print('Total Training Time is {} sec'.format(time.time()-start))

### Checkpoint creation

Disable for now

In [ ]:
# lat = tf.train.latest_checkpoint('../input/traincheckpointceleb/')
# ch = tf.train.Checkpoint()
# ch.restore(lat)

In [ ]:
# # checkpoint.restore(tf.train.latest_checkpoint('../input/traincheckpointceleb/'))
# check = checkpoint.restore(tf.train.latest_checkpoint('../input/traincheckpointceleb/'))

In [ ]:
# !ls ../input/training-checkpoint-celeb-dataset/

## Final image generation and training GIF creation

In [ ]:
def display_image(epoch_no):
    return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('image*.png')
    filenames = sorted(filenames)
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

import tensorflow_docs.vis.embed as embed
embed.embed_file(anim_file)

In [ ]:
generator.load_weights('../input/generator-weights-39/generator_weights_epoch_39')
random = tf.random.normal([1,100])
plt.imshow(tf.reshape(random,(10,10)))
plt.show()
image = generator.predict(random)
plt.imshow(image[0])

In [ ]:
# generator.save_weights('generator_weights_checkpoint')
discriminator.save_weights('discrimininator_weights_checkpoint')